In [1]:
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import tensorflow

Using TensorFlow backend.


In [2]:
# Load the dataset and prepare it to the train the model

# Importing dataset and splitting into words and labels
dataset = pd.read_csv('datasets/intents.csv', names=["Query", "Intent"])

X = dataset["Query"]
y = dataset["Intent"]

unique_intent_list = list(set(y))

# print(queries)
# print(unique_intent_list)
print("Dataset successfully loaded!")

Dataset successfully loaded!


In [3]:
queryCorpus = []
ps = PorterStemmer()

for query in X:
    query = re.sub('[^a-zA-Z]', ' ', query)

    # Tokenize sentence
    query = query.split(' ')

    # Lemmatizing
    tokenized_query = [ps.stem(word.lower()) for word in query]

    # Recreate the sentence from tokens
    tokenized_query = ' '.join(tokenized_query)

    # Add to corpus
    queryCorpus.append(tokenized_query)
print(queryCorpus)
print("Corpus created")

['queri', 'how to grow rice ', 'i want to grow rice ', 'how much water to add for wheat ', 'best season to grow sugarcan', 'yield of wheat', 'averag yield of rice', 'what is the yield of rice in belgaum ', 'best pesticid for cotton ', 'best soil for cotton', 'how much fertil should i add to my rice crop ', 'when to sow cotton ', 'when to harvest wheat ', 'how to enhanc yield of sugarcan ', 'i want to grow paddi ', 'how to grow maiz ', 'which fertil can i use for sugarcan ', 'how is the weather today ', 'how is the weather tomorrow ', 'what s the weather in mysor thi week ', 'weather thi week ', 'weather forecast for the week ', 'how is the climat today ', 'climat condit for the upcom day ', 'weather report for the day ', 'will it rain today ', 'is it go to be sunni thi week ', 'what will the weather be like thi week ', 'can i expect it to be sunni today ', 'which is the season for grow rice ', 'season for grow wheat ', 'how much cotton seed are need for one acr of land ', 'can i grow r

In [4]:
countVectorizer= CountVectorizer(max_features=800)
corpus = countVectorizer.fit_transform(queryCorpus).toarray()
print(corpus.shape)
# print(len(corpus))
print("Bag of words created!")

(129, 162)
Bag of words created!


In [5]:
# Encode the intent classes
labelencoder_intent = LabelEncoder()
y = labelencoder_intent.fit_transform(y)
y = np_utils.to_categorical(y)
print("Encoded the classes!")
print(y)

Encoded the classes!
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
# Return a dictionary, mapping labels to their integer values
res = {}
for cl in labelencoder_intent.classes_:
    res.update({cl:labelencoder_intent.transform([cl])[0]})

intent_label_map = res
print(intent_label_map)
print("Label mapping obtained!")

{'AskingHelp': 0, 'BestPractice': 1, 'Cultivation': 2, 'CultivationSeason': 3, 'Diseases': 4, 'EnquireAboutDay': 5, 'Family': 6, 'Fertilizer': 7, 'Greeting': 8, 'HarvestTime': 9, 'Intent': 10, 'Irrigation': 11, 'Location': 12, 'MarketPrice': 13, 'Maturity': 14, 'OutOfScope': 15, 'Pesticide': 16, 'Rainfall': 17, 'ReapingSeason': 18, 'Sarcasm': 19, 'SeedDensity': 20, 'Soil': 21, 'Varieties': 22, 'Wassup': 23, 'Weather': 24, 'Wellness': 25, 'Yield': 26}
Label mapping obtained!


In [7]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size = 0.15, random_state = 19)

print("Dataset split into train and test set")
print(X_train.shape)
print(y_train.shape)

Dataset split into train and test set
(109, 162)
(109, 27)


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 96, kernel_initializer = 'uniform', activation = 'relu', input_dim = 162))

# Adding the second hidden layer
classifier.add(Dense(units = 96, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 27, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [10]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 500)

Epoch 1/500
109/109 [==============================] - 1s 14ms/step - loss: 3.2937 - acc: 0.1193
Epoch 2/500
109/109 [==============================] - 0s 238us/step - loss: 3.2778 - acc: 0.2294
Epoch 3/500
109/109 [==============================] - 0s 385us/step - loss: 3.2472 - acc: 0.2569
Epoch 4/500
109/109 [==============================] - 0s 367us/step - loss: 3.1751 - acc: 0.2569
Epoch 5/500
109/109 [==============================] - 0s 348us/step - loss: 3.0312 - acc: 0.2477
Epoch 6/500
109/109 [==============================] - 0s 266us/step - loss: 2.8545 - acc: 0.2110
Epoch 7/500
109/109 [==============================] - 0s 422us/step - loss: 2.6685 - acc: 0.1835
Epoch 8/500
109/109 [==============================] - 0s 257us/step - loss: 2.5007 - acc: 0.1927
Epoch 9/500
109/109 [==============================] - 0s 303us/step - loss: 2.3367 - acc: 0.2569
Epoch 10/500
109/109 [==============================] - 0s 275us/step - loss: 2.1501 - acc: 0.3119
Epoch 11/500
109/109

109/109 [==============================] - 0s 312us/step - loss: 0.0046 - acc: 1.0000
Epoch 84/500
109/109 [==============================] - 0s 449us/step - loss: 0.0045 - acc: 1.0000
Epoch 85/500
109/109 [==============================] - 0s 367us/step - loss: 0.0043 - acc: 1.0000
Epoch 86/500
109/109 [==============================] - 0s 248us/step - loss: 0.0042 - acc: 1.0000
Epoch 87/500
109/109 [==============================] - 0s 248us/step - loss: 0.0041 - acc: 1.0000
Epoch 88/500
109/109 [==============================] - 0s 321us/step - loss: 0.0040 - acc: 1.0000
Epoch 89/500
109/109 [==============================] - 0s 257us/step - loss: 0.0038 - acc: 1.0000
Epoch 90/500
109/109 [==============================] - 0s 275us/step - loss: 0.0037 - acc: 1.0000
Epoch 91/500
109/109 [==============================] - 0s 284us/step - loss: 0.0036 - acc: 1.0000
Epoch 92/500
109/109 [==============================] - 0s 440us/step - loss: 0.0035 - acc: 1.0000
Epoch 93/500
109/109 [=

109/109 [==============================] - 0s 284us/step - loss: 8.3456e-04 - acc: 1.0000
Epoch 165/500
109/109 [==============================] - 0s 211us/step - loss: 8.2251e-04 - acc: 1.0000
Epoch 166/500
109/109 [==============================] - 0s 211us/step - loss: 8.1189e-04 - acc: 1.0000
Epoch 167/500
109/109 [==============================] - 0s 275us/step - loss: 8.0017e-04 - acc: 1.0000
Epoch 168/500
109/109 [==============================] - 0s 211us/step - loss: 7.8750e-04 - acc: 1.0000
Epoch 169/500
109/109 [==============================] - 0s 266us/step - loss: 7.7572e-04 - acc: 1.0000
Epoch 170/500
109/109 [==============================] - 0s 238us/step - loss: 7.6507e-04 - acc: 1.0000
Epoch 171/500
109/109 [==============================] - 0s 238us/step - loss: 7.5464e-04 - acc: 1.0000
Epoch 172/500
109/109 [==============================] - 0s 330us/step - loss: 7.4381e-04 - acc: 1.0000
Epoch 173/500
109/109 [==============================] - 0s 266us/step - loss:

109/109 [==============================] - 0s 248us/step - loss: 3.1769e-04 - acc: 1.0000
Epoch 243/500
109/109 [==============================] - 0s 312us/step - loss: 3.1499e-04 - acc: 1.0000
Epoch 244/500
109/109 [==============================] - 0s 220us/step - loss: 3.1196e-04 - acc: 1.0000
Epoch 245/500
109/109 [==============================] - 0s 303us/step - loss: 3.0864e-04 - acc: 1.0000
Epoch 246/500
109/109 [==============================] - 0s 266us/step - loss: 3.0495e-04 - acc: 1.0000
Epoch 247/500
109/109 [==============================] - 0s 266us/step - loss: 3.0162e-04 - acc: 1.0000
Epoch 248/500
109/109 [==============================] - 0s 238us/step - loss: 2.9898e-04 - acc: 1.0000
Epoch 249/500
109/109 [==============================] - 0s 229us/step - loss: 2.9566e-04 - acc: 1.0000
Epoch 250/500
109/109 [==============================] - 0s 238us/step - loss: 2.9221e-04 - acc: 1.0000
Epoch 251/500
109/109 [==============================] - 0s 422us/step - loss:

109/109 [==============================] - 0s 249us/step - loss: 1.5067e-04 - acc: 1.0000
Epoch 321/500
109/109 [==============================] - 0s 238us/step - loss: 1.4935e-04 - acc: 1.0000
Epoch 322/500
109/109 [==============================] - 0s 238us/step - loss: 1.4807e-04 - acc: 1.0000
Epoch 323/500
109/109 [==============================] - 0s 394us/step - loss: 1.4679e-04 - acc: 1.0000
Epoch 324/500
109/109 [==============================] - 0s 266us/step - loss: 1.4554e-04 - acc: 1.0000
Epoch 325/500
109/109 [==============================] - 0s 266us/step - loss: 1.4417e-04 - acc: 1.0000
Epoch 326/500
109/109 [==============================] - 0s 266us/step - loss: 1.4296e-04 - acc: 1.0000
Epoch 327/500
109/109 [==============================] - 0s 238us/step - loss: 1.4203e-04 - acc: 1.0000
Epoch 328/500
109/109 [==============================] - 0s 248us/step - loss: 1.4056e-04 - acc: 1.0000
Epoch 329/500
109/109 [==============================] - 0s 257us/step - loss:

109/109 [==============================] - 0s 248us/step - loss: 7.9275e-05 - acc: 1.0000
Epoch 399/500
109/109 [==============================] - 0s 211us/step - loss: 7.8675e-05 - acc: 1.0000
Epoch 400/500
109/109 [==============================] - 0s 238us/step - loss: 7.8142e-05 - acc: 1.0000
Epoch 401/500
109/109 [==============================] - 0s 266us/step - loss: 7.7441e-05 - acc: 1.0000
Epoch 402/500
109/109 [==============================] - 0s 266us/step - loss: 7.6892e-05 - acc: 1.0000
Epoch 403/500
109/109 [==============================] - 0s 284us/step - loss: 7.6391e-05 - acc: 1.0000
Epoch 404/500
109/109 [==============================] - 0s 330us/step - loss: 7.5702e-05 - acc: 1.0000
Epoch 405/500
109/109 [==============================] - 0s 275us/step - loss: 7.5111e-05 - acc: 1.0000
Epoch 406/500
109/109 [==============================] - 0s 275us/step - loss: 7.4587e-05 - acc: 1.0000
Epoch 407/500
109/109 [==============================] - 0s 248us/step - loss:

109/109 [==============================] - 0s 193us/step - loss: 4.4545e-05 - acc: 1.0000
Epoch 476/500
109/109 [==============================] - 0s 248us/step - loss: 4.4242e-05 - acc: 1.0000
Epoch 477/500
109/109 [==============================] - 0s 367us/step - loss: 4.3952e-05 - acc: 1.0000
Epoch 478/500
109/109 [==============================] - 0s 257us/step - loss: 4.3611e-05 - acc: 1.0000
Epoch 479/500
109/109 [==============================] - 0s 220us/step - loss: 4.3286e-05 - acc: 1.0000
Epoch 480/500
109/109 [==============================] - 0s 229us/step - loss: 4.2990e-05 - acc: 1.0000
Epoch 481/500
109/109 [==============================] - 0s 211us/step - loss: 4.2700e-05 - acc: 1.0000
Epoch 482/500
109/109 [==============================] - 0s 248us/step - loss: 4.2372e-05 - acc: 1.0000
Epoch 483/500
109/109 [==============================] - 0s 275us/step - loss: 4.2032e-05 - acc: 1.0000
Epoch 484/500
109/109 [==============================] - 0s 248us/step - loss:

In [12]:
# Predict the result
y_pred = classifier.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred 

array([ 0,  6, 11,  2,  0, 24, 22, 25, 24, 11, 25,  0,  2,  6, 24, 13,  8,
        6, 21, 25], dtype=int64)

In [13]:
# y_test = np.argmax(y_test, axis=0)
y_test

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],


In [14]:
y_test.shape


(20, 27)

In [15]:
y_pred.shape

(20,)

In [16]:
y_test2 = np.argmax(y_test, axis=1)

In [17]:
y_test2

array([ 0,  6,  3,  2,  0, 24,  0, 25, 24,  7, 25,  0, 12, 19, 24, 13,  8,
       19, 16,  6], dtype=int64)

In [18]:
cm = confusion_matrix(y_test2, y_pred)
cm

array([[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]], dtype=int64)